[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fin-py/option-data-hands-on/blob/main/docs/appendix.ipynb)

# オマケ

jquants-derivativesを活用したWebアプリケーション例

## Panel

https://panel.holoviz.org/

- ダッシュボードなどを作成できるPythonパッケージ
- PyDataエコシステムに対応
    - pandas
    - Jupyter
    - etc

In [1]:
import datetime

import pandas as pd
import panel as pn
import plotly.express as px
from jquants_derivatives import Client, Option, database, plot_volatility

pn.extension("plotly")

### Widgets

https://panel.holoviz.org/reference/index.html#widgets

In [2]:
int_slider = pn.widgets.IntSlider(name="Integer Slider", start=0, end=10, value=4)
int_slider

IntSlider(end=10, name='Integer Slider', value=4)

In [3]:
int_slider.value

4

### Panes

https://panel.holoviz.org/reference/index.html#panes

In [4]:
df = pd.DataFrame([1, 2])
pn.Row(pn.pane.DataFrame(df), pn.pane.Plotly(px.line(df)))

Row
    [0] DataFrame(DataFrame)
    [1] Plotly(Figure)

## jquants-derivativesを活用したWebアプリケーション例

In [5]:
def update(event):
    option = Option(Client().get_option_index_option(str(datetime_picker.value.date())))
    date = option.date
    date_text.value = date
    contract_month = option.contract_month[int_input.value]
    contract_month_text.value = contract_month
    df = option.contracts_dfs[option.contract_month[int_input.value]]
    df_pane.object = df
    fig = plot_volatility(option)
    plot_pane.object = fig


now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
today = now.date()
days_ago = (now - datetime.timedelta(days=7)).date()
calendar = Client().get_markets_trading_calendar(
    holiday_division=1, from_yyyymmdd=days_ago, to_yyyymmdd=today
)
latest = calendar.loc[:, "Date"].iloc[-2]
option = Option(Client().get_option_index_option(str(latest.date())))
df_pane = pn.pane.DataFrame(
    option.contracts_dfs[option.contract_month[0]],
    sizing_mode="stretch_both",
    max_height=400,
    max_width=1000,
)
fig = plot_volatility(option)
plot_pane = pn.pane.Plotly(fig)


date_text = pn.widgets.StaticText(name="Date", value=option.date)
contract_month_text = pn.widgets.StaticText(
    name="Contract Month", value=option.contract_month[0]
)
int_input = pn.widgets.IntInput(name="marurity", value=0, step=1, start=0, end=1)
datetime_picker = pn.widgets.DatetimePicker(name="Date", value=latest)
update_button = pn.widgets.Button(name="update")
update_button.on_click(update)

app = pn.Column(
    pn.Row(date_text, contract_month_text),
    pn.Row(datetime_picker, int_input, update_button),
    df_pane,
    plot_pane,
    height=1200,
    width=1200,
)
app.servable()

Column(height=1200, sizing_mode='fixed', width=1200)
    [0] Row
        [0] StaticText(name='Date', value=Timestamp('2023-10-11 0...)
        [1] StaticText(name='Contract Month', value='2023-10')
    [1] Row
        [0] DatetimePicker(name='Date', value=Timestamp('2023-10-11 0...)
        [1] IntInput(end=1, name='marurity', start=0)
        [2] Button(name='update')
    [2] DataFrame(DataFrame, max_height=400, max_width=1000, sizing_mode='stretch_both')
    [3] Plotly(Figure)